In [127]:
import pandas as pd
import numpy as np

In [156]:
CARDS = {
    '2': 2, '3': 3, '4': 4, '5': 5,
    '6': 6, '7': 7, '8': 8, '9': 9,
    'T': 10, 'J': 11, 'Q': 12, 'K': 13, 'A': 14
}

STRAIGHTS = {
    tuple(range(i,i+5)) for i in range(2,11)
}
STRAIGHTS.add((2,3,4,5,14))

In [129]:
def card_sort(c):
    return CARDS[c[0]]

In [130]:
def val_cnts(hand):
    cnts = {}
    for card in hand:
        cnts[card[0]] = cnts.get(card[0], 0) + 1
    
    return sorted(cnts.items(), key = lambda a: (a[1], CARDS[a[0]]), reverse = True)

In [271]:
def high_card(hand):
    return tuple(sorted([CARDS[card[0]] for card in hand], reverse = True))

In [272]:
def high_card_check(hand1, hand2, separator = ""):
    hc1 = high_card(hand1)
    hc2 = high_card(hand2)

    for i in range(min(len(hc1), len(hc2))):
        if hc1[i] > hc2[i]:
            return 1, f"{separator}high card {hc1[i]}"
        elif hc1[i] < hc2[i]:
            return 2, f"{separator}high card {hc2[i]}"

    return 0, "tie"

In [273]:
def pair(hand):
    cnts = val_cnts(hand)
    return cnts[0][1] == 2, CARDS[cnts[0][0]]

In [274]:
def two_pair(hand):
    cnts = val_cnts(hand)
    return cnts[0][1] == 2 and cnts[1][1] == 2, CARDS[cnts[0][0]]

In [275]:
def trips(hand):
    cnts = val_cnts(hand)
    return cnts[0][1] == 3, CARDS[cnts[0][0]]

In [276]:
def straight(hand):
    vals = tuple(sorted([CARDS[card[0]] for card in hand]))
    return vals in STRAIGHTS, vals[-1] if max(vals) < 14 else 5

In [277]:
def flush(hand):
    vals = tuple(sorted([CARDS[card[0]] for card in hand]))
    return len(set(card[1] for card in hand)) == 1, vals[-1]

In [278]:
def full_house(hand):
    t, hc = trips(hand)
    p, _ = pair(hand)
    return t and p, hc

In [279]:
def quads(hand):
    cnts = val_cnts(hand)
    return cnts[0][1] == 4, CARDS[cnts[0][0]]

In [280]:
def straight_flush(hand):
    s, hc = straight(hand)
    f, _ = flush(hand)
    return s and f, hc

In [281]:
def royal_flush(hand):
    sf, _ = straight_flush(hand)
    hcs = high_card(hand)
    return sf and hcs[0] == 14 and hcs[1] == 13, 14

In [282]:
def poker_hand_check(hand1, hand2):
    print("hand1:", sorted(hand1, key = card_sort))
    print("hand2:", sorted(hand2, key = card_sort))

    if len(hand1) < 5 or len(hand2) < 5:
        raise ValueError("hands must have 5 cards")

    for func in (royal_flush, straight_flush, quads, full_house, flush, straight, trips, two_pair, pair):
        check1, hc1 = func(hand1)
        check2, hc2 = func(hand2)

        if check1 and not check2:
            return 1, func.__name__
        elif check2 and not check1:
            return 2, func.__name__
        elif check1 and check2:
            if hc1 > hc2:
                return 1, func.__name__, f"high card {hc1}"
            elif hc1 < hc2:
                return 2, func.__name__, f"high card {hc1}"
    
    return high_card_check(hand1,hand2)

In [283]:
with open('../../inputs/0054_poker.txt', 'r') as f:
    hand_str = f.read()
    hands = hand_str.split("\n")
    hands1 = [h[:14].split() for h in hands]
    hands2 = [h[15:].split() for h in hands]

In [285]:
win_cnt = {0: 0, 1: 0, 2: 0}
winners1 = []
for i in range(1000):
    winner, *descriptors = poker_hand_check(hands1[i], hands2[i])
    win_cnt[winner] += 1
    winners1.append(winner)
    print(winner, ",", descriptors)
    print()

print(win_cnt)

hand1: ['4S', '8C', '9H', 'TS', 'KC']
hand2: ['2S', '3S', '5D', '7D', 'AC']
2 , ['high card 14']

hand1: ['5C', '5D', '9C', 'AD', 'AC']
hand2: ['5H', '7C', '8D', 'TD', 'KS']
1 , ['two_pair']

hand1: ['3H', '6S', '7H', 'JS', 'KC']
hand2: ['2D', '8S', 'TD', 'JC', 'QH']
1 , ['high card 13']

hand1: ['5C', '8H', 'TH', 'TC', 'QS']
hand2: ['4D', '9H', 'JC', 'JS', 'KS']
2 , ['pair', 'high card 10']

hand1: ['5H', '7C', 'JD', 'QH', 'KC']
hand2: ['4C', '4S', 'KH', 'AS', 'AD']
2 , ['two_pair']

hand1: ['5H', '7D', '9C', '9H', 'KS']
hand2: ['3S', '5D', '5C', '8D', 'AH']
1 , ['pair', 'high card 9']

hand1: ['2H', '3H', '4H', '5C', '6H']
hand2: ['3S', '5S', '6S', 'QH', 'AS']
1 , ['straight']

hand1: ['4H', '7C', '8C', 'TD', 'TC']
hand2: ['3H', '4C', '7S', 'KC', 'KS']
2 , ['pair', 'high card 10']

hand1: ['3H', '6D', '7C', '9C', 'KD']
hand2: ['4C', 'QS', 'QC', 'KH', 'AC']
2 , ['pair']

hand1: ['2H', '2D', '5H', '6S', 'JC']
hand2: ['7C', '9D', 'JS', 'KD', 'AS']
1 , ['pair']

hand1: ['8H', '9D', 'TH',